In [1]:
from pynq import Overlay
from pynq import allocate 
import numpy as np
import struct
import subprocess
import time
import re

In [2]:
#PARAMETERS
MAX_ROT_GROUP_SIZE = 128
MAX_KSI_POWS_SIZE = 513
MAX_QPOW_VEC_SIZE = 65
SIGMA = 3.2
H = 64 
RND_SIZE = 257
LOG_N = 8
LOG_Q = 32
LOG_q = 32
LOG_P = 30
MAX_Z_SIZE = 256
MAX_ZZX_DEGREE = 513
MAX_SLOTS = 8
slots = np.int64(8)

In [3]:
rnd_filename = "data/rnd.data"

secret_key_coeff_filename = "data/secretKey.data"
secret_key_degree_filename = "data/secretKey.degree"

enc_key_ax_coeff_filename = "data/encKeyAx.data"
enc_key_ax_degree_filename = "data/encKeyAx.degree"

enc_key_bx_coeff_filename = "data/encKeyBx.data"
enc_key_bx_degree_filename = "data/encKeyBx.degree"

message_filename = "data/message.data"

cipher_ax_coeff_filename = "data/cipherAx.data"
cipher_ax_degree_filename = "data/cipherAx.degree"

cipher_bx_coeff_filename = "data/cipherBx.data"
cipher_bx_degree_filename = "data/cipherBx.degree"



data_generator_path = './data_gen'

In [4]:
# Execute the binary for data generation
result = subprocess.run([data_generator_path], capture_output=True, text=True)

# Check the return code to see if the execution was successful
if result.returncode == 0:
    # Print the output of the binary
    print("Output stdout:", result.stdout)
else:
    # Print the error if the binary was not executed successfully
    print("Error executing the binary")
    print("Output stderr:", result.stderr)

output = result.stdout
encryption_time_match = re.search(r'\[INFO\] SW encryption execution time: (\d+) microseconds', output)
decryption_time_match = re.search(r'\[INFO\] SW decryption execution time: (\d+) microseconds', output)

if encryption_time_match and decryption_time_match:
    encryption_time = int(encryption_time_match.group(1))
    decryption_time = int(decryption_time_match.group(1))

Output stdout: context initiazlied
Context Parameters : 
sigma: 3.2
h: 64
N: 256
Nh: 128
logNh: 7
M: 512
logQQ: 64
Q: 4294967296
QQ: 18446744073709551616
secret key initialized
scheme initialized
[INFO] SW encryption execution time: 877180 microseconds
[INFO] SW decryption execution time: 423265 microseconds



In [5]:
def read_long_array_from_file(filename, size):
    # read long array from binary file
    data = np.fromfile(filename, dtype=np.int64, count=size)
    return data

In [6]:
def read_int64_from_file(filename):
    # Reads a single 64-bit integer from the binary file
    data = np.fromfile(filename, dtype=np.int64, count=1)
    
    # Checks if the file contains at least one 64-bit integer
    if data.size == 0:
        raise ValueError(f"The file {filename} is empty or does not contain a valid 64-bit integer.")
    
    return data[0]

In [7]:
def read_and_convert_array(filename, size, max_z_size):
    # Calculates the number of int64 needed to represent an integer of max_z_size bits
    num_int64 = max_z_size // 64
    
    # Reads the data from the binary file as integers of max_z_size bits
    data = np.fromfile(filename, dtype=np.int64, count=size*num_int64)
    
    return data

In [8]:
def read_complex_array_from_file(filename, size):
    # Reads the data from the binary file as 128-bit complex numbers (complex128)
    data = np.fromfile(filename, dtype=np.complex128, count=size)

    return data

In [9]:
rnd_array = read_long_array_from_file(rnd_filename, RND_SIZE)
print(f"Size of the array in bytes: {rnd_array.nbytes}")

Size of the array in bytes: 2056


In [10]:
sx_degree = read_int64_from_file(secret_key_degree_filename)
print(f"Secret key degree: {sx_degree}")

Secret key degree: 256


In [11]:
sx = read_and_convert_array(secret_key_coeff_filename, sx_degree +1, MAX_Z_SIZE)
print(f"Size of the secret key in bytes: {sx.nbytes}")

Size of the secret key in bytes: 8224


In [12]:
ax_degree = read_int64_from_file(enc_key_ax_degree_filename)
print(f"EncKey.ax degree: {ax_degree}")

ax = read_and_convert_array(enc_key_ax_coeff_filename, ax_degree + 1, MAX_Z_SIZE)
print(f"Size of EncKey.ax in bytes: {ax.nbytes}")

EncKey.ax degree: 256
Size of EncKey.ax in bytes: 8224


In [13]:
bx_degree = read_int64_from_file(enc_key_bx_degree_filename)
print(f"EncKey.bx degree: {bx_degree}")

bx = read_and_convert_array(enc_key_bx_coeff_filename, bx_degree + 1, MAX_Z_SIZE)
print(f"Size of EncKey.bx in bytes: {bx.nbytes}")

EncKey.bx degree: 256
Size of EncKey.bx in bytes: 8224


In [14]:
message_array = read_complex_array_from_file(message_filename, slots)
print(f"Size of the message array in bytes: {message_array.nbytes}")

Size of the message array in bytes: 128


In [15]:
overlay = Overlay("./ckks_encryption_wrapper.xsa")

In [16]:
enc_ip = overlay.ckks_ker_0

In [17]:
enc_ip.register_map.CTRL

Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0)

In [18]:
enc_ip.read(0x00)

4

In [19]:
buf_rnd = allocate(RND_SIZE, np.int64)

buf_rnd[:] = rnd_array[:]
buf_rnd.flush()


buf_sx = allocate((sx_degree+1)*(MAX_Z_SIZE//64), np.int64)

buf_sx[:] = sx[:]
buf_sx.flush()


buf_enc_ax = allocate((ax_degree+1)*(MAX_Z_SIZE//64), np.int64)

buf_enc_ax[:] = ax[:]
buf_enc_ax.flush()

buf_enc_bx = allocate((bx_degree+1)*(MAX_Z_SIZE//64), np.int64)
buf_enc_bx[:] = bx[:]
buf_enc_bx.flush()

buf_message = allocate(slots, np.complex128)
buf_message[:] = message_array[:]
buf_message.flush()

In [20]:
buf_ax_out = allocate((MAX_ZZX_DEGREE)*(MAX_Z_SIZE//64), np.int64)
buf_bx_out = allocate((MAX_ZZX_DEGREE)*(MAX_Z_SIZE//64), np.int64)
buf_ax_degree_out = allocate(1,np.int64)
buf_bx_degree_out = allocate(1,np.int64)

In [21]:
slots_bytes = slots.tobytes()
ax_degree_b = ax_degree.tobytes()
bx_degree_b = bx_degree.tobytes()

In [22]:
enc_ip.write(0x10,buf_message.physical_address)
enc_ip.write(0x1c,slots_bytes[:4])
enc_ip.write(0x20,slots_bytes[-4:])
enc_ip.write(0x28,buf_enc_ax.physical_address)
enc_ip.write(0x34,ax_degree_b[:4])
enc_ip.write(0x38,ax_degree_b[-4:])
enc_ip.write(0x40,buf_enc_bx.physical_address)
enc_ip.write(0x4c,bx_degree_b[:4])
enc_ip.write(0x4c,bx_degree_b[-4:])
enc_ip.write(0x58,buf_rnd.physical_address)
enc_ip.write(0x64,buf_ax_out.physical_address)
enc_ip.write(0x70,buf_bx_out.physical_address)
enc_ip.write(0x7c,buf_ax_degree_out.physical_address)
enc_ip.write(0x88,buf_bx_degree_out.physical_address)

In [23]:

enc_ip.write(0x00,1)
start_time = time.perf_counter()
while(enc_ip.read(0x00) & 0x04 != 0x04):
      pass
end_time = time.perf_counter()

In [24]:
# Calculate the execution time in microseconds
execution_time_us = (end_time - start_time) * 1_000_000

In [25]:
buf_ax_degree_out.invalidate()
buf_bx_degree_out.invalidate()

buf_ax_out.invalidate()
buf_bx_out.invalidate()

In [26]:
ci_ax_degree = read_int64_from_file(cipher_ax_degree_filename)
print(f"Cipher.ax degree: {ci_ax_degree}")

ci_ax = read_and_convert_array(cipher_ax_coeff_filename, ci_ax_degree + 1, MAX_Z_SIZE)
print(f"Size of Cipher.ax in bytes: {ci_ax.nbytes}")

Cipher.ax degree: 256
Size of Cipher.ax in bytes: 8224


In [27]:
ci_bx_degree = read_int64_from_file(cipher_bx_degree_filename)
print(f"Cipher.bx degree: {ci_bx_degree}")

ci_bx = read_and_convert_array(cipher_bx_coeff_filename, ci_bx_degree + 1, MAX_Z_SIZE)
print(f"Size of Cipher.bx in bytes: {ci_bx.nbytes}")

Cipher.bx degree: 256
Size of Cipher.bx in bytes: 8224


In [28]:
if(buf_ax_degree_out != ci_ax_degree):
    print("ERROR polynomial ax degree different from software")

In [29]:
if(buf_bx_degree_out != ci_bx_degree):
    print("ERROR polynomial bx degree different from software")

In [30]:
for i in range (0, (ci_ax_degree+1)*(MAX_Z_SIZE//64)):
    if(buf_ax_out[i]!=ci_ax[i]):
        print("ERROR POLYNOMIAL AX DIFFERENT FROM HW AT INDEX: ",i)

In [31]:
for i in range (0, (ci_bx_degree+1)*(MAX_Z_SIZE//64)):
    if(buf_bx_out[i]!=ci_bx[i]):
        print("ERROR POLYNOMIAL BX DIFFERENT FROM HW AT INDEX: ",i)

In [32]:
print("HW EXECUTION TIME: ", execution_time_us, " microseconds")
print("SW EXECUTION TIME: ", encryption_time, " microseconds")
print("HW SPEEDUP: ", encryption_time/execution_time_us, "x")

HW EXECUTION TIME:  422.157347202301  microseconds
SW EXECUTION TIME:  877180  microseconds
HW SPEEDUP:  2077.8508435615327 x
